# Install

⭐Before running anything, click 'Runtime' on menu bar, then click 'change runtime type'. Make sure Hardware Accelerator is set to GPU.

⭐Mounting your google drive👇. Run the block, it will show you link for authorisation. Go to the link location, then allow access, copy the code that pops up, paste it at “Enter your authorization code:”

In [0]:
from google.colab import drive 
drive.mount('/gdrive') 

⭐Run these three code blocks once. (Click on each block, and then click the ▶ button on the left side of the selected block)

In [0]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [0]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [0]:
!pip install -r requirements.txt

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_cyclegan_dataset.sh [apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]`

Or use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

⭐Run the block below to download your dataset👇. change 'horse2zebra' with the dataset you are training.

In [0]:
!bash ./datasets/download_cyclegan_dataset.sh horse2zebra

# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


⭐ During training, the program will periodically save checkpoints to your google drive in cyclegan/horse2zebra/checkpoints/ folder.
Replace horse2zebra in all the paths with the name of the dataset you're training. Run the block below👇 to start training.

In [0]:
!python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan --checkpoints_dir /gdrive/cyclegan/horse2zebra/checkpoints --n_epochs 50

⭐You can now close the window. The training will proceed in the cloud. But please check in periodically to ensure that it is still running (You can either go to the link of your notebook, or click runtime->manage sessions, and you will see your active session there). 

⭐If by chance your session ended/disconnected, You will have to run all the blocks in the 'Install' section (cloning git, mounting google drive etc.), download the dataset from the 'Datasets' section, and then run the block below to continue training👇(remember to replace the horse2zebra with your dataset name):

In [0]:
!python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan --checkpoints_dir /gdrive/cyclegan/horse2zebra/checkpoints --n_epochs 50 --continue_train

⭐I haven't looked into Testing and Visualizing. They'll need a bit of tweaking. Will fix later.

# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [0]:
!python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout

# Visualize

In [0]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_fake.png')
plt.imshow(img)

In [0]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_real.png')
plt.imshow(img)